In [1]:
import re, os, sys, glob
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn
from sklearn.linear_model import LinearRegression
from matplotlib import pyplot as plt
from fractions import Fraction

In [2]:
def clean_column_names(self):
    new_column_names = {old:re.sub( string=old.lower()
                                   ,pattern='\W' #\W matches non-alphnumeric
                                   ,repl='_')\
                              .strip('_')
                        for old in self.columns
                       }
    return (self.rename(columns=new_column_names))
pd.DataFrame.clean_column_names = clean_column_names

In [3]:
def parse_date_columns(self):
    for date_column in self.filter(regex=("date")).columns:
        self[date_column] = pd.to_datetime(self[date_column])
    return self
pd.DataFrame.parse_date_columns = parse_date_columns

In [4]:
def zero_to_null(self,subset):
    for column in subset:
        self[column] = self[column].apply(lambda x: x if x != 0 else np.nan)
    return self
pd.DataFrame.zero_to_null = zero_to_null

In [5]:
def merge_multi(self, df, on):
    return self.reset_index().merge(df.reset_index(),on=on,how='left').set_index(self.index.names)

pd.DataFrame.merge_multi = merge_multi

In [6]:
def deduplicate(self,key,NUMERIC='max',NON_NUMERIC='first',override={}):
    how_to_agg = {index: NUMERIC if np.issubdtype(value,np.number) else NON_NUMERIC 
                  for (index, value) in self.dtypes.iteritems()
                 }
    how_to_agg.update(override)
    return self.groupby(key).agg(how_to_agg)
pd.DataFrame.deduplicate = deduplicate

In [7]:
def parse_api_columns(self):
    for api_column in self.filter(regex=("api")).columns:
        self[api_column] = self[api_column].apply(str)\
                                           .str.replace('\W','')\
                                           .str.pad( 14
                                                    ,side='right'
                                                    ,fillchar='0'
                                                   )
    return self
pd.DataFrame.parse_api_columns = parse_api_columns 

In [8]:
casing = pd.concat(map( pd.read_csv
                       ,glob.glob('./data/welldatabase/casing/*.csv')
                      ))\
           .clean_column_names()\
           .parse_date_columns()\
           .parse_api_columns()

AttributeError: 'DataFrame' object has no attribute 'clean_columns'

In [ ]:
completion = pd.concat(map( pd.read_csv
                       ,glob.glob('./data/welldatabase/completion/*.csv')
                      ))\
           .clean_column_names()\
           .parse_date_columns()\
           .parse_api_columns()
        

In [ ]:
directional = pd.concat(map( pd.read_csv
                       ,glob.glob('./data/welldatabase/directional/*.csv')
                      ))\
           .clean_column_names()\
           .parse_date_columns()\
           .parse_api_columns()
        

In [ ]:
formation = pd.concat(map( pd.read_csv
                       ,glob.glob('./data/welldatabase/formation/*.csv')
                      ))\
           .clean_column_names()\
           .parse_date_columns()\
           .parse_api_columns()
        

In [ ]:
fracstage = pd.concat(map( pd.read_csv
                       ,glob.glob('./data/welldatabase/fracstage/*.csv')
                      ))\
           .clean_column_names()\
           .parse_date_columns()\
           .parse_api_columns()
        

In [ ]:
header = pd.concat(map( pd.read_csv
                       ,glob.glob('./data/welldatabase/header/*.csv')
                      ))\
           .clean_column_names()\
           .parse_date_columns()\
           .parse_api_columns()
        

In [ ]:
perf = pd.concat(map( pd.read_csv
                       ,glob.glob('./data/welldatabase/perf/*.csv')
                      ))\
           .clean_column_names()\
           .parse_date_columns()\
           .parse_api_columns()

In [ ]:
production = pd.concat(map( pd.read_csv
                       ,glob.glob('./data/welldatabase/production/*.csv')
                      ))\
           .clean_column_names()\
           .parse_date_columns()\
           .parse_api_columns()

In [ ]:
productionsummary = pd.concat(map( pd.read_csv
                       ,glob.glob('./data/welldatabase/productionsummary/*.csv')
                      ))\
           .clean_column_names()\
           .parse_date_columns()\
           .parse_api_columns()
        

In [ ]:
test = pd.concat(map( pd.read_csv
                       ,glob.glob('./data/welldatabase/test/*.csv')
                      )
                  )\
           .clean_column_names()\
           .parse_date_columns()\
           .parse_api_columns()

In [ ]:
def string_to_fraction(x):
    try: 
        result = float(Fraction(x))
    except AttributeError:
        result = float(x)
    except TypeError:
        result = np.nan
    except ValueError:
        result = np.nan
    while result > 1:
        result = result / 64
    return result


In [ ]:
test['chokesize_clean'] = test['chokesize'].str.replace('-','/')\
                                           .str.replace('TH','')\
                                           .str.strip('"')\
                                           .str.strip("'")\
                                           .str.replace(pat="OPEN|NONE|FO|OPEN FLOW",repl='1')\
                                           .str.replace(pat="0|CLOSED|INSERT",repl='')\
                                           .apply(string_to_fraction)

In [ ]:
tubingandpacker = pd.concat(map( pd.read_csv
                       ,glob.glob('./data/welldatabase/tubingandcasing/*.csv')
                      ))\
           .clean_columns()\
           .parse_date_columns()

In [ ]:
header = header[header['wellboreprofile']=='HORIZONTAL']
sqdist = (header['surfacelatitude']-header['bottomholelatitude'])**2 + (header['surfacelongitude']-header['bottomholelongitude'])**2
header['surface_to_bottomhole_distance'] = sqdist.map(lambda x: np.sqrt(x))

In [ ]:
linear_reg = LinearRegression()
xy=header.loc[header['surface_to_bottomhole_distance']!=0,['laterallength','surface_to_bottomhole_distance']].dropna()
y = xy['laterallength'].values.reshape(-1, 1)
X = xy['surface_to_bottomhole_distance'].values.reshape(-1, 1)
linear_reg.fit(X=X,y=y)

In [ ]:
plt.plot( X ,y ,'o'
         ,X,linear_reg.predict(X),'-k')

In [ ]:
header['laterallength_from_bottom'] = linear_reg.predict(header['surface_to_bottomhole_distance'].values.reshape(-1, 1))

In [ ]:
header['missing_laterallength'] =  header['laterallength'].isnull()

In [ ]:
header_summary = header.deduplicate('api',override={'missing_laterallength':'all'})

In [ ]:
completion = zero_to_null(completion,['upperperf','lowerperf'])

In [ ]:
completion_summary = completion.deduplicate(['api','completiondate'])

In [ ]:
index_of_deepest_top = formation.groupby("api").agg({'topdepth':'idxmax'})['topdepth'].dropna()
formation_summary = formation.iloc[index_of_deepest_top].groupby('api').agg({'name':'first'})

In [ ]:
fracstage_summary = fracstage.deduplicate('api')

In [ ]:
perf = zero_to_null(perf,['lowerperf','upperperf'])

In [ ]:
perf_summary = perf.deduplicate('api')

In [ ]:
production['yearmonth'] = production['date'].apply(lambda x: '{YEAR}-{MONTH:02d}'.format(YEAR=x.year,MONTH=x.month))

In [ ]:
production = production[production['yearmonth'] > '2011-01']

In [ ]:
production['days'] = pd.to_timedelta(production['days'],unit='D')
production['first_producing_day_of_month'] = production['date'] + pd.DateOffset(months=1) - production['days']

In [ ]:
production_summary = production.deduplicate(['api','yearmonth'])

In [ ]:
test_summary = test.deduplicate(['api','testdate'])

In [ ]:
productionsummary_summary = productionsummary.deduplicate('api')

In [ ]:
combined = header_summary.join(productionsummary_summary)\
                         .join(perf_summary)\
                         .join(fracstage_summary)\
                         .merge_multi(test_summary,on='api')\
                         .merge_multi(completion_summary,on='api')\
                         .merge_multi(production_summary,on='api')

In [ ]:
combined['days_since_completion'] = ( combined['first_producing_day_of_month'] - combined['completiondate'] ) / np.timedelta64(1, 'D')

In [ ]:
combined['increment_30days'] = combined['days_since_completion'].apply(lambda x: np.floor(x / 30) + 1)

In [ ]:
combined = combined[combined['first_producing_day_of_month'] >= combined['completiondate']]
combined.shape

In [ ]:
combined_subset = combined[combined['increment_30days'].map(lambda x: 0 <= x <= 12)]
combined_subset.shape

In [ ]:
combined_subset = combined_subset.deduplicate(key=['api','increment_30days'],override={'oil':'sum'})
combined_subset.shape

In [ ]:
combined_subset['increment_30days'].value_counts()

In [ ]:
combined_subset.to_csv('./data/welldb_combined.csv')